In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

#
# Model pipeline
#
D = 2 ** 10

features = [str(i) for i in range(1,40)]


fields = ['id','label'] + numeric_features + categorical_features + ["day_number"]

# Now we have a full prediction pipeline.
model = Pipeline(steps=[
    ('logisticregression', LogisticRegression())
])

def get_x(csv_row, D):
    x = [0]  # 0 is the index of the bias term
    for key, value in csv_row.items():
        
        index = int(str(value) + key[1:], 16) % D  # weakest hash ever ;)
        x.append(index)
    return x  # x contains indices of features that have a value of 1

In [ ]:
import os, sys
import logging

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump
from sklearn.metrics import log_loss

In [ ]:
train_path='/home/users/datasets/criteo_train1'

In [ ]:
X_train = []
Y_train = []
X_test = []
Y_test = []

for t, row in enumerate(open(train_path)):
    if t%10==0:
        if t%3==0:
            row = dict(zip(fields, row.replace("\n",'').split('\t')))
            y = 1. if row['label'] == '1' else 0.
            
            del row['label']  # can't let the model peek the answer
            del row['id']  # we don't need the Id
            del row['day_number']

            # step 1, get the hashed features
            x = get_x(row, D)
            x_row = np.zeros(D)
            x_row[x] = 1
            X_test.append(x_row)
            Y_test.append(y)
        else:
            row = dict(zip(fields, row.replace("\n",'').split('\t')))
            y = 1. if row['label'] == '1' else 0.

            del row['label']  # can't let the model peek the answer
            del row['id']  # we don't need the Id
            del row['day_number']

            # step 1, get the hashed features
            x = get_x(row, D)
            x_row = np.zeros(D)
            x_row[x] = 1
            X_train.append(x_row)
            Y_train.append(y)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
read_opts=dict(
        sep='\t', names=fields, index_col=False, header=None, nrows=100, keep_default_na=False
)

In [ ]:
df = pd.read_csv(train_path, **read_opts)

In [ ]:
df.drop(columns = ['label'], inplace=True)

In [ ]:
df.head()

In [ ]:
pred = []

In [ ]:
for index, row in df.iterrows():
    test=row

In [ ]:
del row['id']  # we don't need the Id
del row['day_number']

In [ ]:
D = 2 ** 10

In [ ]:
x = get_x(row, D)
x_row = np.zeros(D)
x_row[x] = 1
pred.append(model.predict(x_row.reshape(1, -1))[0])

In [ ]:
pred

In [ ]:
read_opts=dict(
        sep='\t', names=fields, index_col=False, header=None, nrows=100, keep_default_na=False
)
df = pd.read_csv(train_path, **read_opts)
df.drop(columns = ['label'], inplace=True)

pred = []
for index, row in df.iterrows():
    ids = df['id']
    del row['id']  # we don't need the Id
    del row['day_number']
    x = get_x(row, D)
    x_row = np.zeros(D)
    x_row[x] = 1
    pred.append(model.predict(x_row.reshape(1, -1))[0]) 
out = zip(ids, pred)

In [ ]:
print("\n".join(["{0},{1}".format(*i) for i in out]))

In [ ]:
#!/opt/conda/envs/dsenv/bin/python

import sys, os
import logging
from joblib import load
import pandas as pd

sys.path.append('.')
from model import D, get_x

#
# Init the logger
#
logging.basicConfig(level=logging.DEBUG)
logging.info("CURRENT_DIR {}".format(os.getcwd()))
logging.info("SCRIPT CALLED AS {}".format(sys.argv[0]))
logging.info("ARGS {}".format(sys.argv[1:]))

#load the model
model = load("1.joblib")

numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)]
fields = ["id"] + numeric_features + categorical_features + ["day_number"]

#read and infere
read_opts=dict(
        sep='\t', names=fields, index_col=False, header=None,
        iterator=True, chunksize=1000, keep_default_na=False
)

for df in pd.read_csv(sys.stdin, **read_opts):
    pred = []
    for index, row in df.iterrows():
        ids = df['id']
        del row['id']  # we don't need the Id
        del row['day_number']
        x = get_x(row, D)
        x_row = np.zeros(D)
        x_row[x] = 1
        pred.append(model.predict(x_row.reshape(1, -1))[0])
    out = zip(ids, pred)
    print("\n".join(["{0},{1}".format(*i) for i in out]))

In [1]:
from joblib import load
import pandas as pd
import numpy as np
from model import D, get_x

numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)]
fields = ["id"] + numeric_features + categorical_features + ["day_number"]

#read and infere
read_opts=dict(
        sep='\t', names=fields, index_col=False, header=None,
        iterator=True, chunksize=1000, keep_default_na=False
)

df = pd.read_csv('/home/users/ValentinMSU17/ozon-masters-bigdata/test', **read_opts)

model = load("/home/users/ValentinMSU17/ozon-masters-bigdata/1.joblib")

In [2]:
for df in pd.read_csv('/home/users/ValentinMSU17/ozon-masters-bigdata/test', **read_opts):
    pred = []
    for index, row in df.iterrows():
        ids = df['id']
        del row['id']  # we don't need the Id
        del row['day_number']
        x = get_x(row, D)
        x_row = np.zeros(D)
        x_row[x] = 1
        pred.append(model.predict(x_row.reshape(1, -1))[0])
    out = zip(ids, pred)
    print("\n".join(["{0},{1}".format(*i) for i in out]))

0,0.0
1,0.0
2,0.0
3,0.0


In [ ]:
test

In [5]:
!head /home/users/datasets/criteo_valid1_labels

0	0
1	0
2	0
3	0
4	0
5	0
6	0
7	0
8	0
9	0


In [11]:
predict_true = pd.read_csv('/home/users/datasets/criteo_valid1_labels', sep='\t', header=None, index_col=False)

In [12]:
predict_true

,0,1
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
8033453,8033453,1
8033454,8033454,1
8033455,8033455,1
8033456,8033456,1


In [8]:
predict = pd.read_csv('/home/users/ValentinMSU17/ozon-masters-bigdata/predict_result.csv', sep=',', header=None, index_col=False)

In [13]:
np.sum(predict['1'])

KeyError: '1'

In [18]:
np.sum(predict_true.iloc[:,1])

266584

In [19]:
predict

,0,1
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
8033453,8033453,0.0
8033454,8033454,0.0
8033455,8033455,0.0
8033456,8033456,0.0
